In [1]:
import os
os.environ["YOLO_X_MODEL_PATH"]=r"C:/Users/Zhi-F/.cache/huggingface/hub/yolox/yolox_l0.05.onnx"

In [2]:
from unstructured.partition.pdf import partition_pdf
import base64
import os
output_path=r"c:/Users/Zhi-F/repo/hugging_face/multimodel_rag/extracted_images/"
pdf_path=r"c:/Users/Zhi-F/repo/hugging_face/content/attention.pdf"

os.environ["UNSTRUCTURED_PARALLELIZE"] = "True"  # 开启多线程
os.environ["UNSTRUCTURED_THREADS"] = "12"  # 线程数=CPU核心数（如4/8，根据自己电脑调整）

# 使用partition_pdf函数将PDF文档分解为可处理的文本块
chunks=partition_pdf(filename=pdf_path,          # PDF文件的完整路径
                     infer_table_structure=True,  # 是否自动识别和提取PDF中的表格结构
                     strategy="hi_res",           # PDF解析策略："hi_res"(高精度)，"fast"(快速)，或"ocr_only"(仅OCR)
                     extract_image_block_types=["Image","Table"],  # 提取的图像块类型列表，如["Image"]表示提取所有图片
                     extract_image_block_output_dir=output_path,  # 图像块的保存目录
                     extract_image_block_to_payload=True,  # 是否将提取的图像块作为数据载荷包含在结果中
                     chunking_strategy="by_title",  # 文本分块策略："by_title"(按标题分块)，"by_page"(按页面分块)，或"basic"
                     #max_characters=10000,         # 每个文本块的最大字符数限制，超过则进一步分块
                     #combine_text_under_n_chars=2000,  # 合并小文本块的阈值：小于此长度的文本块会被合并到相邻块
                     #new_after_n_chars=6000)       # 强制新分块的字符数：当累计字符数达到此值时强制开始新分块
                    )


C:\Users\Zhi-F\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [3]:
chunks[0].metadata.orig_elements

In [4]:
tables = []
texts = []
for chunk in chunks:
    if "Table" in str(type(chunk)):
        tables.append(chunk)
    if "CompositeElement" in str(type(chunk)):
        texts.append(chunk)
print(len(tables))
print(len(texts))


4
105


In [5]:
images = []

def get_images_base64(chunks):
    images_base64=[]
    for chunk in chunks:
        if "CompositeElement" in str(type(chunk)):
            elements= chunk.metadata.orig_elements
            for el in elements:
                if 'Image' in str(type(el)):
                    images_base64.append(el.metadata.image_base64)
    return images_base64

images= get_images_base64(chunks)

In [6]:
print(len(texts))
print(len(images))
print(len(tables))
tables[0].to_dict()

105
7
4


{'type': 'Table',
 'element_id': '58246ac1d674122055fa1f9b8e56318d',
 'text': 'Layer Type Complexity per Layer Sequential Maximum Path Length Operations Self-Attention O(n2 · d) O(1) O(1) Recurrent O(n · d2) O(n) O(n) Convolutional O(k · n · d2) O(1) O(logk(n)) Self-Attention (restricted) O(r · n · d) O(1) O(n/r)',
 'metadata': {'last_modified': '2025-11-09T20:08:10',
  'text_as_html': '<table><tr><td>Layer Type</td><td>Complexity per Layer</td><td>Sequential Operations</td><td>Maximum Path Length</td></tr><tr><td>Self-Attention</td><td>O(n? - d)</td><td>O(1)</td><td>O(1)</td></tr><tr><td>Recurrent</td><td>O(n-d?)</td><td>O(n)</td><td>O(n)</td></tr><tr><td>Convolutional</td><td>O(k-n-d?)</td><td>olny</td><td>O(logx(n))</td></tr><tr><td>Self-Attention (restricted)</td><td>O(r-n-d)</td><td>ol)</td><td>O(n/r)</td></tr></table>',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 6,
  'orig_elements': '...',
  'file_directory': 'c:/Users/Zhi-F/repo/hugging_face/conte

In [ ]:
import base64
from IPython.display import Image

def display_image_base64(image_base64):
    image_data = base64.b64decode(image_base64)
    return Image(data=image_data)

display_image_base64(images[0])

In [11]:
from  langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

os.environ["DEEPSEEK_API_KEY"] = "sk-2b53bca28369400ca20dcd08b904332b"
llm_deepseek = ChatOpenAI(
            model_name="deepseek-chat",
            openai_api_base="https://api.deepseek.com/v1",
            openai_api_key=os.environ["DEEPSEEK_API_KEY"],
            temperature=0.3,  # 降低温度以提高准确性
            request_timeout=60,  # 核心：设置合理超时
            max_retries=1,  # 添加重试机制
            callbacks=[],  # 禁用所有回调
        )

sys_prompt= """
You are an assistant tasked with summarizing tables and text.
Give a concise summary of the table or text.

Respond only with the summary, no additionnal comment.
Do not start your message by saying "Here is a summary" or anything like that.
Just give the summary as it is.

Table or text chunk: {element}

"""
template= ChatPromptTemplate.from_messages([
    ("system", sys_prompt),
    ("human", "{element}")
])

summarize_chain = (template | llm_deepseek | StrOutputParser())
text_sum=summarize_chain.batch(texts, config={"max_concurrency": 5})
table_html=[table.metadata.text_as_html for table in tables]
table_sum=summarize_chain.batch(table_html, config={"max_concurrency": 5})

In [12]:
print(text_sum[0])
print(table_sum[0])
print(images[0])

The paper "Attention Is All You Need" by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, and Łukasz Kaiser was published in 2017 on arXiv.
Self-Attention has O(n²·d) complexity, O(1) sequential operations, and O(1) path length. Recurrent layers have O(n·d²) complexity, O(n) sequential operations, and O(n) path length. Convolutional layers have O(k·n·d²) complexity, O(1) sequential operations, and O(logₖ(n)) path length. Restricted Self-Attention has O(r·n·d) complexity, O(1) sequential operations, and O(n/r) path length.
...


In [13]:
import base64
import requests
import json



In [14]:

def img_to_text(base64_image)->str:

    url = "https://dashscope.aliyuncs.com/api/v1/services/aigc/multimodal-generation/generation"
        
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer sk-ab7bb97028bf47e9a2607ba684f587ee"
    }
    prompt="Describe the image in detail. For context, the image is part of a research paper explaining the transformers architecture. Be specific about graphs, such as bar plots."
    payload = {
        "model": "qwen-vl-max",  # 或 "qwen-vl-plus", "qwen-vl-v1"
        "input": {
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "image": f"data:image/jpeg;base64,{base64_image}"
                        },
                            {
                                "text": prompt
                            }
                        ]
                    }
                ]
            },
            "parameters": { "max_tokens": 1500  }
        }

    response = requests.post(url, headers=headers, data=json.dumps(payload))
    result = response.json()

    if "output" in result and "choices" in result["output"]:
        description = result["output"]["choices"][0]["message"]["content"]
    else:
        description = f"Exception: {result}"

    return description

In [15]:
images_sum=[]
for base64_image in images:
    text=img_to_text(base64_image)
    images_sum.append(text)

In [16]:
images_sum[0]

[{'text': 'The image is a **diagrammatic representation of the Transformer architecture**, which is a foundational model in natural language processing (NLP) introduced in the paper *"Attention is All You Need"* by Vaswani et al. This architecture revolutionized sequence modeling by replacing recurrent layers with self-attention mechanisms.\n\n---\n\n### **Overall Structure**\n\nThe diagram illustrates the **encoder-decoder architecture** of the Transformer, consisting of two main components:\n\n1. **Encoder Stack** (on the left)\n2. **Decoder Stack** (on the right)\n\nEach stack contains multiple identical layers repeated **N times** (indicated by "Nx"), where N is typically 6 in standard implementations.\n\n---\n\n### **Input and Output Flow**\n\n- **Inputs** are fed into the **Input Embedding** layer (pink box), which converts token IDs into dense vectors.\n- These embeddings are then combined with **Positional Encoding** (represented as a circle with a spiral pattern), which inject

In [17]:
import uuid
from langchain_community.vectorstores import Chroma
from langchain_core.stores import InMemoryStore
from langchain_core.documents import Document
from langchain_core.embeddings import Embeddings
from langchain_classic.retrievers.multi_vector import MultiVectorRetriever
from langchain_huggingface.embeddings import HuggingFaceEmbeddings


local_model_path = r"C:\Users\Zhi-F\.cache\huggingface\hub\models--BAAI--bge-small-zh-v1.5\snapshots\\7999e1d3359715c523056ef9478215996d62a620"
embeddings = HuggingFaceEmbeddings(
    model_name=local_model_path
)


In [81]:
chroma=Chroma(collection_name="attention",embedding_function=embeddings, persist_directory="./chroma_db")
# 删除整个集合
chroma.delete_collection()

In [82]:
chroma=Chroma(collection_name="attention",embedding_function=embeddings, persist_directory="./chroma_db")
doc_store=InMemoryStore()
id_key="doc_id"  #metadata

#init retriever, only retrive data from doc_store not vectorstore
retriever=MultiVectorRetriever(
    vectorstore=chroma,
    docstore=doc_store,
    id_key=id_key,
)



In [72]:
doc_ids=[str(uuid.uuid4()) for _ in range(len(text_sum))]
print(len(doc_ids))
text_sum_docs=[Document(page_content=summary,metadata={id_key:doc_ids[i]}) for i,summary in enumerate(text_sum)]
retriever.vectorstore.add_documents(text_sum_docs) #把文字的摘抄存入向量数据库
retriever.docstore.mset(list(zip(doc_ids,texts))) #把文字内容存入文档数据库



105


In [70]:
tables_ids=[str(uuid.uuid4()) for _ in range(len(table_sum))]
print(len(tables_ids))
print(table_sum[0])
table_sum_docs=[Document(page_content=summary,metadata={id_key:tables_ids[i]}) for i,summary in enumerate(table_sum)]
retriever.vectorstore.add_documents(table_sum_docs) #把表格的摘抄存入向量数据库
retriever.docstore.mset(list(zip(tables_id,table_sum))) #把表格内容存入文档数据库


4
Self-Attention has O(n²·d) complexity, O(1) sequential operations, and O(1) path length. Recurrent layers have O(n·d²) complexity, O(n) sequential operations, and O(n) path length. Convolutional layers have O(k·n·d²) complexity, O(1) sequential operations, and O(logₖ(n)) path length. Restricted Self-Attention has O(r·n·d) complexity, O(1) sequential operations, and O(n/r) path length.


In [94]:
from typing import Any


img_ids=[str(uuid.uuid4()) for _ in range(len(images_sum))]
print(len(img_ids))
print(images[0])
img_sum_docs = [
    Document(
        page_content=summary[0]['text'] if summary and isinstance(summary, list) and len(summary) > 0 else "",
        metadata={id_key: img_ids[i]}
    ) 
    for i, summary in enumerate(images_sum)
]
retriever.vectorstore.add_documents(img_sum_docs) #把图片的摘抄存入向量数据库
retriever.docstore.mset(list(zip(img_ids, images))) #把图片内容存入文档数据库

7
...


In [86]:
chroma.persist()

In [95]:
docs=retriever.invoke("who are the authors of the paper",
     id_key='doc_id',
    search_kwargs={'k': 1} 
)


In [96]:
for doc in docs:
    print(str(doc) + "\n\n" + "-" * 80)

...

--------------------------------------------------------------------------------
...

--------------------------------------------------------------------------------
...

--------------------------------------------------------------------------------
...

--------------------------------------------------------------------------------
